# Homework 2  

The goal of this assignment is experiment with classification pipelines (in this case, for instrument classification) using spectrograms.

In [30]:
!rm -rf nsynth_subset

In [31]:
# Comment this out when submitting
!tar -xvzf nsynth_subset.tar.gz

nsynth_subset/
nsynth_subset/guitar_acoustic_010-044-100.wav
nsynth_subset/vocal_synthetic_003-075-050.wav
nsynth_subset/guitar_acoustic_015-094-075.wav
nsynth_subset/vocal_synthetic_003-087-100.wav
nsynth_subset/guitar_acoustic_030-041-075.wav
nsynth_subset/vocal_synthetic_003-034-050.wav
nsynth_subset/guitar_acoustic_030-021-025.wav
nsynth_subset/guitar_acoustic_021-080-025.wav
nsynth_subset/vocal_acoustic_000-059-075.wav
nsynth_subset/vocal_synthetic_003-099-050.wav
nsynth_subset/guitar_acoustic_014-106-075.wav
nsynth_subset/guitar_electronic_028-108-100.wav
nsynth_subset/vocal_synthetic_003-072-075.wav
nsynth_subset/guitar_acoustic_015-107-075.wav
nsynth_subset/guitar_acoustic_015-036-075.wav
nsynth_subset/vocal_synthetic_003-039-075.wav
nsynth_subset/vocal_acoustic_000-068-127.wav
nsynth_subset/guitar_acoustic_014-047-025.wav
nsynth_subset/vocal_acoustic_000-059-025.wav
nsynth_subset/guitar_electronic_028-026-075.wav
nsynth_subset/vocal_acoustic_000-066-050.wav
nsynth_subset/vocal

In [8]:
# Set this yourself depending where you put the files
# dataroot = "/home/julian/Downloads/"
dataroot = "."
# On the autograder it should be here:
# dataroot = "."

In [9]:
# !pip install librosa
# !pip install torch
# !pip install glob
# !pip install numpy

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import librosa
import random
import glob

In [11]:
torch.use_deterministic_algorithms(True) # Try to make things less random, though not required

In [12]:
audio_paths = glob.glob(dataroot + "/nsynth_subset/*.wav")
random.seed(0)
random.shuffle(audio_paths)

In [13]:
if not len(audio_paths):
    print("You probably need to set the dataroot folder correctly")

In [14]:
SAMPLE_RATE = 8000 # Very low sample rate, just so things run quickly
N_MFCC = 13
INSTRUMENT_MAP = {'guitar': 0, 'vocal': 1} # Only two classes (also so that things run quickly)
NUM_CLASSES = len(INSTRUMENT_MAP)

# If we used all the classes we would have:
# INSTRUMENT_MAP = {
#     'bass': 0, 'brass': 1, 'flute': 2, 'guitar': 3,
#     'keyboard': 4, 'mallet': 5, 'organ': 6, 'reed': 7,
#     'string': 8, 'synth_lead': 9, 'vocal': 10
# }

1. Extract prediction labels and construct waveforms

`extract_waveform()`

**Inputs**
- `path`: A string that represents a path to the wav file

**Outputs**
- `waveform`: an array containing the waveform; use librosa.load, remember to set the sample rate correctly

`extract_label()`

**Inputs**
- `path'

**Outputs**
- `label`: A integer that represents the label of the path (hint: look at the filename and make use of `INSTRUMENT_MAP`)

In [15]:
def extract_waveform(path):
    # Your code here
    waveform, _ = librosa.load(path, sr=SAMPLE_RATE)
    return waveform

In [16]:
def extract_label(path):
    # Your code here
    filename = path[path.rfind('/')+1:]  # eg. 'guitar_acoustic_010-044-100.wav' from 'nsynth_subset/guitar_acoustic_010-044-100.wav'
    instrument_name = filename.split("_")[0]  # eg. 'guitar' from 'guitar_acoustic_010-044-100.wav'
    label = INSTRUMENT_MAP[instrument_name]
    return label

In [17]:
waveforms = [extract_waveform(p) for p in audio_paths]
labels = [extract_label(p) for p in audio_paths]

In [18]:
# sanity check
print(waveforms[0])
print(labels[0])

[-2.8059129e-03 -9.8028034e-03  1.2120705e-02 ... -3.2764859e-05
 -5.5099372e-06  8.2980841e-07]
1


A few simple classifiers are provided. You don't need to modify these (though the autograder will *probably* work if you'd like to experiment with architectural changes)

In [19]:
class MLPClassifier(nn.Module):
    def __init__(self):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(2 * N_MFCC, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, NUM_CLASSES)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [20]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(64, NUM_CLASSES)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(nnF.relu(self.bn1(self.conv1(x))))
        x = self.pool2(nnF.relu(self.bn2(self.conv2(x))))
        x = self.pool3(nnF.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

2. Extract mfcc features

`extract_mfcc()`

**Inputs**
- `waveform`: an array containing the waveform

**Outputs**
- `feature`: a PyTorch float tensor that represents a concatenation of 13 mean values and 13 standard deviation values

**Process**
- Extract feature using `librosa.feature.mfcc`; remember to set the sample rate and n_mfcc
- Compute 13 mean and 13 standard deviation values
- Concatenate them together

In [25]:
def extract_mfcc(w):
    # Your code here:
    # load using librosa.feature.mfcc
    mfcc = librosa.feature.mfcc(y=w, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
    # extract mean and std
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_std = mfcc.std(axis=1)
    # concatenate
    features = np.concatenate([mfcc_mean, mfcc_std])
    return torch.FloatTensor(features)

## Note:

The autograder will test that your MFCC features are correct, and it will *also* use them within an ML pipeline. The test_suite can be used to run the full pipeline after you've implemented these functions. If you've implemented your features correctly this should "just work" and you'll be able to upload the trained; this is mostly here just so that you can see how the full pipeline works (which will be useful when you develop your own pipelines for Assignment 1)

3. Extract spectrograms

`extract_spec()`

**Inputs**
- `waveform`: an array containing the waveform

**Outputs**
- `feature`: a PyTorch float tensor that contains a spectrogram

**Process**
- apply STFT to the given waveform
- square the absolute values of the complex numbers from the STFT

In [33]:
def extract_spec(w):
    # Your code here
    # load
    stft = librosa.stft(y=w)
    # take squared absolute values
    spec = np.abs(stft) ** 2
    feature = torch.from_numpy(spec).float()
    return feature

4. Extract mel-spectrograms

`extract_mel()`

**Inputs**
- `waveform`: an array containing the waveform
- `n_mels`: number of mel bands
- `hop_length`: hop length

**Outputs**
- `feature`: A PyTorch Float Tensor that contains a mel-spectrogram

**Process**
- generate melspectrograms with `librosa.feature.melspectrogram`; make sure to se the sample rate, n_mels, and hop_length
- convert them to decibel units with `librosa.power_to_db`
- normalize values to be in the range 0 to 1

In [40]:
def extract_mel(w, n_mels = 128, hop_length = 512):
    # Your code here
    # load
    mel_spec = librosa.feature.melspectrogram(y=w, sr=SAMPLE_RATE, n_mels=n_mels, hop_length=hop_length)
    # convert to db
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    # normalize
    mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min())
    return torch.FloatTensor(mel_spec_db)

5. Extract constant-Q transform

`extract_q()`

**Inputs**
- `waveform`: an array containing the waveform

**Outputs**
- `feature`: A PyTorch Float Tensor that contains a constant-Q transform

**Process**
- generate constant-Q transform with `librosa.cqt`; this one will need a higher sample rate (use 16000) to work

In [45]:
def extract_q(w):
    # Your code here
    result = np.abs(librosa.cqt(w, sr=16000))
    return torch.FloatTensor(result)

6. Pitch shift

`pitch_shift()`

**Inputs**
- `waveform`: an array containing the waveform
- `n`: number of semitones to shift by (integer, can be positive or negative)

**Outputs**
- `waveform`: a pitch-shifted waveform

**Process**
- use `librosa.effects.pitch_shift`

In [43]:
def pitch_shift(w, n):
    # Your code here
    y_shift = librosa.effects.pitch_shift(y=w, sr=SAMPLE_RATE, n_steps=n)
    return y_shift

In [ ]:
# Code below augments the datasets

augmented_waveforms = []
augmented_labels = []

for w,y in zip(waveforms,labels):
    augmented_waveforms.append(w)
    augmented_waveforms.append(pitch_shift(w,1))
    augmented_waveforms.append(pitch_shift(w,-1))
    augmented_labels += [y,y,y]

7. Extend the model to work for four classes.

By making data augmentations, or modifying the model architecture, build a model with test accuracy > 0.93

In [72]:
INSTRUMENT_MAP_7 = {'guitar_acoustic': 0, 'guitar_electronic': 1, 'vocal_acoustic': 2, 'vocal_synthetic': 3}

In [73]:
NUM_CLASSES_7 = 4

In [84]:
def extract_label_7(path):
    # Your code here
    filename = path[path.rfind('/')+1:]  # eg. 'guitar_acoustic_010-044-100.wav' from 'nsynth_subset/guitar_acoustic_010-044-100.wav'
    instrument_name = filename[:filename.rfind('_')]  # eg. 'guitar' from 'guitar_acoustic_010-044-100.wav'
    label = INSTRUMENT_MAP_7[instrument_name]
    return label

In [86]:
# Select which feature function to use.
# Can be one of the existing ones (e.g. extract_mfcc), or you can write a new one.
def feature_func_7(w):
    # # load using librosa.feature.mfcc
    # mfcc = librosa.feature.mfcc(y=w, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
    # # extract mean and std
    # mfcc_mean = mfcc.mean(axis=1)
    # mfcc_std = mfcc.std(axis=1)

    # # This didn't give me high enough accuracy
    # # # extract spectral_bandwidth mean and std
    # # spec_bandwidth = librosa.feature.spectral_bandwidth(y=w, sr=SAMPLE_RATE)
    # # # kind of a dumb way of doing this but I need the feature vectors to be of length N_MFCC
    # # spec_bw = np.zeros(N_MFCC)
    # # spec_bw[0] = spec_bandwidth.mean()
    # # spec_bw[1] = spec_bandwidth.std()
    # # # extract spec_rolloff mean and std
    # # spec_rolloff = librosa.feature.spectral_rolloff(y=w, sr=SAMPLE_RATE)
    # # spec_ro = np.zeros(N_MFCC)
    # # spec_ro[0] = spec_rolloff.mean()
    # # spec_ro[1] = spec_rolloff.std()
    # # # concatenate
    # # features = np.concatenate([mfcc_mean, mfcc_std, spec_bw, spec_ro])

    # # This didn't work either
    # # Nvm this worked after I changed my model architecture
    # # Try delta mean and std to capture temporal dynamics
    # mfcc_delta = librosa.feature.delta(mfcc)
    # mfcc_delta_mean = mfcc_delta.mean(axis=1)
    # mfcc_delta_std = mfcc_delta.std(axis=1)
    # # concatenate
    # features = np.concatenate([mfcc_mean, mfcc_std, mfcc_delta_mean, mfcc_delta_std])

    # return torch.FloatTensor(features)
    return extract_mfcc(w)

In [87]:
labels_7 = [extract_label_7(p) for p in audio_paths]

In [92]:
class MLPClassifier_4classes(nn.Module):
    # def __init__(self):
    #     super(MLPClassifier_4classes, self).__init__()
    #     self.fc1 = nn.Linear(4 * N_MFCC, 64)
    #     # self.dropout1 = nn.Dropout(0.2)
    #     self.fc2 = nn.Linear(64, 32)
    #     # self.dropout2 = nn.Dropout(0.2)
    #     self.fc3 = nn.Linear(32, NUM_CLASSES_7)
    #     self.relu = nn.ReLU()

    # def forward(self, x):
    #     # x = self.dropout1(self.relu(self.fc1(x)))
    #     # x = self.dropout2(self.relu(self.fc2(x)))
    #     x = self.relu(self.fc1(x))
    #     x = self.relu(self.fc2(x))
    #     x = self.fc3(x)
    #     return x

    def __init__(self):
        super(MLPClassifier_4classes, self).__init__()
        self.fc1 = nn.Linear(2 * N_MFCC, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, NUM_CLASSES_7)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [93]:
# Select which model to use.
# Can use an existing model (e.g. MLPClassifier) or modify it.
# Note that you'll need to copy and (slightly) modify the existing class to handle 4 labels.
model_7 = MLPClassifier_4classes()